## Model 2


Additions to the model (Marten has unsolvable problems with the Julia kernel :/  )

In [76]:
using JuMP,HiGHS,DataFrames


In [77]:
#I=1:3                                  #number of generation units
#Q=1:1                                  #number of stochastic generation units
#T=1:1                                  #number of time intervalls 
#Ω=1:2                                  #number of scenarios
#J=1:2                                  #number of loads
#Λ=1:1                                  #number of buses
#C=[100, 80, 50]                         #unit cost of generation
#C_RU=[20, 10, 15]                       #cost of up reserve commitment
#C_RD=[25, 15, 15]                       #cost of down reserve commitment
#C_U=[10, 15, 20]                        #cost of up reserve commitment
#C_D=[30, 10, 15]                        #cost of down reserve commitment
#π=[0.7, 0.3]                           #probabilities of the scenarios
#V_LOL=[20, 25]                          #value of the lost load
#C_cur=[15]                              #cost of curtailment at turbine
#L= 90 .+ 30 * randn(2, 1)               #loads for every time intervall
#W_realized = 70 .+ 15 * randn(1, 2, 1)  #create random scenarios for four time points
#P_max=[70, 80, 150];                    #maximum production capacity

N = 1:2                                #number of nodes (buses)
I=[[1:2]; [1:1]]                           #number of generation units of node n
# Q [0:0] 
# changed Q
Q = [[1], []]                          # Q[n] gives the wind units at node n
T=1:1                                  #number of time intervalls 
Ω=1:2                                  #number of scenarios
J=[[1:1]; [1:1]]                           #number of loads  of node n
Λ=[[2:2]; [1:1]]                            #set of nodes directly connected to node n
#Λ=[[2] [1]]                            #set of nodes directly connected to node n
C=[[10, 30], [35]]                          #unit cost of generation C[N,I]
C_RU=[[16, 13], [10]]                       #cost of up reserve commitment C_RU[N,I]
C_RD=[[15, 12], [9]]                        #cost of down reserve commitment C_RD[N,I]
C_U=[[10, 30], [35]]                        #cost of up reserve commitment C_U[N,I]
C_D=[[10, 30], [35]]                        #cost of down reserve commitment C_D[N,I]
π=[0.6, 0.4]                           #probabilities of the scenarios
V_LOL=[[200], [200]]                        #value of the lost load
C_cur=[[0]] #[15]                         #cost of curtailment at turbine C_cur[N,Q]
L= [[40], [100]]               #loads for every time intervall L=[N] [J,T]
#W_realized = [[[50] [10]] [[50] [10]] [[50] [10]]] #create random scenarios for four time points q,ω,t

# changed back to original
W_realized = [[[[50],[10]],[]],[]] #70 .+ 15 * randn(1, 1, 2, 1)  #create random scenarios for n,q,ω,t [[[[50],[10]]],[]]#
#[[[111 112] [121 122]] [[211 212] [221 222]] [[311 312] [321 322]]]
P_max=[[50, 110], [100]];                   #maximum production capacity

# From IEEE 24 bus
b = [[1, 1/0.13], [1/0.13, 1]]                     #line susceptance (loss of transmission)
LC_Max = [[0, 100], [100, 0]]          #maximum transmission quantity between two buses [n][ℓ]
print(W_realized[1][1][2][1])

10

In [78]:
data[:node_i][:I]

data= Dict(
    :N => N,
    :I => I,
    :Q => Q,
    :T => T,
    :Ω => Ω,
    :J => J,
    :Λ => Λ,
    :C => C,
    :C_RU => C_RU,
    :C_RD => C_RD,
    :C_U => C_U,
    :C_D => C_D,
    :π => π,
    :V_LOL => V_LOL,
    :C_cur => C_cur,
    :L => L,
    :W_realized => W_realized,
    :P_max => P_max,
    :b => b,
    :LC_Max => LC_Max
    );
#print(W_realized[1,2,1])

Dict{Symbol, AbstractVector} with 20 entries:
  :C_RU       => [[16, 13], [10]]
  :b          => [[1.0, 7.69231], [7.69231, 1.0]]
  :C_U        => [[10, 30], [35]]
  :C_D        => [[10, 30], [35]]
  :P_max      => [[50, 110], [100]]
  :C_cur      => [[0]]
  :LC_Max     => [[0, 100], [100, 0]]
  :Ω          => 1:2
  :Λ          => UnitRange{Int64}[2:2, 1:1]
  :W_realized => Vector{Any}[[Any[[50], [10]], Any[]], []]
  :N          => 1:2
  :C_RD       => [[15, 12], [9]]
  :Q          => Vector{Any}[[1], []]
  :V_LOL      => [[200], [200]]
  :T          => 1:1
  :I          => UnitRange{Int64}[1:2, 1:1]
  :π          => [0.6, 0.4]
  :J          => UnitRange{Int64}[1:1, 1:1]
  :L          => [[40], [100]]
  :C          => [[10, 30], [35]]

In [79]:
function init_variables(model::JuMP.Model)
    P=@variable(model, P[n in data[:N], i in data[:I][n], t in data[:T]] >=0) #energy generated
    R_U=@variable(model, R_U[n in data[:N], i in data[:I][n], t in data[:T]] >=0) #committed upward reserve capacity of generator i
    R_D=@variable(model, R_D[n in data[:N], i in data[:I][n], t in data[:T]] >=0) #committed downward reserve capacity of generator i
    r_U=@variable(model, r_U[n in data[:N], i in data[:I][n], ω in data[:Ω], t in data[:T]] >=0) #up regulation of generator i in case 𝜔
    r_D=@variable(model, r_D[n in data[:N], i in data[:I][n], ω in data[:Ω], t in data[:T]] >=0) #down regulation of generator i in case 𝜔
    L_Shed=@variable(model, L_Shed[n in data[:N], j in data[:J][n], ω in data[:Ω], t in data[:T]] >=0) #loss of load at load demand j in case 𝜔
    W_spill=@variable(model, W_spill[n in data[:N], q in data[:Q][n], ω in data[:Ω], t in data[:T]] >=0) #curtailment of turbine q in case 𝜔
    W_s=@variable(model, W_s[n in data[:N], q in data[:Q][n], t in data[:T]] >=0) #scheduled wind power generation at turbine q
    𝛿=@variable(model, 𝛿[n in data[:N], ℓ in data[:Λ][n], t in data[:T]] >= 0) #voltage angle
    PF=@variable(model, PF[n in data[:N], ℓ in data[:Λ][n], t in data[:T]] >= 0) #power flow          # added >= 0 
    PF_scenario=@variable(model, PF_scenario[ω in data[:Ω], n in data[:N], ℓ in data[:Λ][n], t in data[:T]] >= 0) #power flow scenario
    
    vars= Dict(
        :P => P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s,            # added commas
        :𝛿 => 𝛿,
        :PF => PF,
        :PF_scenario => PF_scenario
    )
    
    return vars
end;


In [80]:
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
            c1[n in data[:N],i in data[:I][n], t in data[:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[:P_max][n][i]
            c2[n in data[:N],i in data[:I][n] ,t in data[:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
            c3[n in data[:N],i in data[:I][n], t in data[:T], ω in data[:Ω]], vars[:r_U][n, i, ω, t] <= vars[:R_U][n,i,t]
            c4[n in data[:N],i in data[:I][n], t in data[:T], ω in data[:Ω]], vars[:r_D][n, i, ω, t] <= vars[:R_D][n,i,t]
            
            c5[ω in data[:Ω], n in data[:N], ℓ in data[:Λ][n], t in data[:T]], vars[:PF_scenario][ω, n, ℓ, t] <= data[:LC_Max][n][ℓ]
            
            c6[n in data[:N], ℓ in data[:Λ][n], t in data[:T]], vars[:PF][n, ℓ, t] <= data[:LC_Max][n][ℓ]
            # Something wrong with powerflow
            c7[ω in data[:Ω], n in data[:N], ℓ in data[:Λ][n], t in data[:T]], (vars[:𝛿][n, ℓ, t] - vars[:𝛿][ℓ, n, t]) * data[:b][n][ℓ] == vars[:PF][n, ℓ, t]

            ## !!
            Power_balance_day_ahead[n in data[:N], t in data[:T]], 
            sum(vars[:P][n,i,t] for i in data[:I][n]) + sum(vars[:W_s][n,q,t] for q in data[:Q][n]) - sum(data[:L][n][j,t] for j in data[:J][n]) - sum(vars[:PF][n, ℓ, t] for ℓ in data[:Λ][n]) == 0

            Power_balance_at_stage[ω in data[:Ω], n in data[:N], t in data[:T]], 
            sum(vars[:r_U][n,i, ω, t] for i in data[:I][n]) -
            sum(vars[:r_D][n,i, ω, t] for i in data[:I][n]) +
            sum(vars[:L_Shed][n,j, ω, t] for j in data[:J][n]) +
            # n ! and W_realized and Q changed
            sum(data[:W_realized][n][q][ω][t] -vars[:W_s][n,q,t] - vars[:W_spill][n,q,ω,t] for q in data[:Q][n]) -
            sum(vars[:PF_scenario][ω, n, ℓ, t] for ℓ in data[:Λ][n]) +
            sum(vars[:PF][n, ℓ, t] for ℓ in data[:Λ][n]) == 0 
    end)
end;


In [81]:
#W_realized[1,1,1,1]
#print(vars[:W_s][1,1,1])
#println(data[:Q])
#println(keys(vars[:W_s]))
data[:b][1][2]


7.692307692307692

In [86]:
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[:C][n][i] + vars[:R_U][n,i,t] * data[:C_RU][n][i] + vars[:R_D][n,i,t] * data[:C_RD][n][i] for n in data[:N], i in data[:I][n], t in data[:T]) +
    sum(π[ω] * (sum(data[:C_U][n][i] * vars[:r_U][n,i, ω ,t] + data[:C_D][n][i] * vars[:r_D][n,i, ω, t] for i in data[:I][n]) +
                sum(data[:C_cur][n][q] * vars[:W_spill][n,q, ω, t] for q in data[:Q][n]) +
                sum(data[:V_LOL][n][j] * vars[:L_Shed][n, j, ω, t] for j in data[:J][n])) for n in data[:N], ω in data[:Ω], t in data[:T])
    )
end;

In [87]:
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
print(model)

Min 10 P[1,1,1] + 16 R_U[1,1,1] + 15 R_D[1,1,1] + 30 P[1,2,1] + 13 R_U[1,2,1] + 12 R_D[1,2,1] + 35 P[2,1,1] + 10 R_U[2,1,1] + 9 R_D[2,1,1] + 6 r_U[1,1,1,1] + 6 r_D[1,1,1,1] + 18 r_U[1,2,1,1] + 18 r_D[1,2,1,1] + 120 L_Shed[1,1,1,1] + 4 r_U[1,1,2,1] + 4 r_D[1,1,2,1] + 12 r_U[1,2,2,1] + 12 r_D[1,2,2,1] + 80 L_Shed[1,1,2,1] + 21 r_U[2,1,1,1] + 21 r_D[2,1,1,1] + 120 L_Shed[2,1,1,1] + 14 r_U[2,1,2,1] + 14 r_D[2,1,2,1] + 80 L_Shed[2,1,2,1]
Subject to
 c7[1,1,2,1] : 7.692307692307692 𝛿[1,2,1] - 7.692307692307692 𝛿[2,1,1] - PF[1,2,1] == 0
 c7[1,2,1,1] : -7.692307692307692 𝛿[1,2,1] + 7.692307692307692 𝛿[2,1,1] - PF[2,1,1] == 0
 c7[2,1,2,1] : 7.692307692307692 𝛿[1,2,1] - 7.692307692307692 𝛿[2,1,1] - PF[1,2,1] == 0
 c7[2,2,1,1] : -7.692307692307692 𝛿[1,2,1] + 7.692307692307692 𝛿[2,1,1] - PF[2,1,1] == 0
 Power_balance_day_ahead[1,1] : P[1,1,1] + P[1,2,1] + W_s[1,1,1] - PF[1,2,1] == 40
 Power_balance_day_ahead[2,1] : P[2,1,1] - PF[2,1,1] == 100
 Power_balance_at_stage[1,1,1] : r_U[1,1,1,1] + r_U[1,2

In [84]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
21 rows, 25 cols, 58 nonzeros
9 rows, 13 cols, 25 nonzeros
Presolve : Reductions: rows 9(-25); columns 13(-23); elements 25(-63)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     3.5000000000e+03 Pr: 3(100) 0s
          4     3.8000000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  3.8000000000e+03
HiGHS run time      :          0.00


In [85]:
# Print the values of the variables
println("Variable values:")
println("P:")
for n in data[:N], i in data[:I], t in data[:T]
    println("P[$i]: ", JuMP.value.(vars[:P][n, i, t]))
end

println("\nR_U:")
for i in data[:I], t in data[:T]
    println("R_U[$i]: ", value(vars[:R_U][i, t]))
end

println("\nR_D:")
for i in data[:I], t in data[:T]
    println("R_D[$i]: ", value(vars[:R_D][i, t]))end

println("\nL_Shed:")
for j in data[:J], ω in data[:Ω], t in data[:T]
    println("L_Shed[$j, $ω, $t]: ", value(vars[:L_Shed][j, ω, t]))
end

println("\nW_spill:")
for q in data[:Q], ω in data[:Ω], t in data[:T]
    println("W_spill[$q, $ω, $t]: ", value(vars[:W_spill][q, ω, t]))
end

println("\nW_realized:")
for q in data[:Q], ω in data[:Ω], t in data[:T]
    println("W_realized[$q, $ω, $t]: ", value(data[:W_realized][q, ω, t]))
end

println("\nW_s:")
for q in data[:Q], t in data[:T]
    println("W_s[$q, $t]: ", value(vars[:W_s][q, t]))
end

println("\nr_U:")
for i in data[:I], ω in data[:Ω], t in data[:T]
    println("r_U[$i, $ω, $t]: ", value(vars[:r_U][i, ω, t]))
end

println("\nr_D:")
for i in data[:I], ω in data[:Ω], t in data[:T]
    println("r_D[$i, $ω, $t]: ", value(vars[:r_D][i, ω, t]))
end

println("\nLoad")
for j in data[:J]
    print("L[$j]:", data[:L][j])
end

print("\n\nsum:", sum(data[:L]))

Variable values:
P:
P[1:2]:   [1]  =  30.0
  [2]  =  0.0
P[1:1]:   [1]  =  30.0
P[1:2]:   [1]  =  100.0
P[1:1]:   [1]  =  100.0

R_U:


BoundsError: BoundsError: attempt to access JuMP.Containers.SparseAxisArray{VariableRef, 3, Tuple{Int64, Int64, Int64}} with 3 entries at index [1:2, 1]